In [29]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy.optimize import curve_fit, OptimizeWarning
from tqdm import tqdm
import warnings
from scipy.stats import zscore
from statsmodels.tsa.stattools import acf, pacf
from scipy.optimize import minimize
from vqr import VectorQuantileRegressor
from vqr.solvers.regularized_lse import RegularizedDualVQRSolver

sns.set_theme()
sns.set_context("notebook")
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
dtype_dict = {
    'FarmName_Pseudo': 'str',
    'SE_Number': 'str',
    'AnimalNumber': 'Int64',          
    'StartDate': 'str',
    'StartTime': 'str',
    'DateTime': 'str',
    'LactationNumber': 'Int64',       
    'DaysInMilk': 'Int64', 
    'YearSeason': 'str',           
    'TotalYield': 'float',
    'DateTime': 'str',
    'BreedName': 'str',
    'Age': 'Int64',
    'Mother': 'str',
    'Father': 'str',
    'CullDecisionDate': 'str',
    'Temperature': 'float',
    'RelativeHumidity': 'float',      
    'THI_adj': 'float',
    'HW': 'Int64',                    
    'cum_HW': 'Int64',                
    'Temp15Threshold': 'Int64'        
}


# Load the CSV with specified dtypes
data = pd.read_csv('../Data/MergedData/CleanedYieldData.csv', dtype=dtype_dict)

# Convert date and time columns back to datetime and time objects
data['DateTime'] = pd.to_datetime(data['DateTime'], errors='coerce')
data['StartTime'] = pd.to_datetime(data['StartTime'], format='%H:%M:%S', errors='coerce').dt.time
data['StartDate'] = pd.to_datetime(data['StartDate'], errors='coerce')
data['CullDecisionDate'] = pd.to_datetime(data['CullDecisionDate'], errors='coerce')
data['DateTime'] = pd.to_datetime(data['DateTime'], errors='coerce')
data.head()

,FarmName_Pseudo,SE_Number,AnimalNumber,StartDate,StartTime,LactationNumber,DaysInMilk,TotalYield,DateTime,YearSeason,...,Mother,Father,CullDecisionDate,Temperature,RelativeHumidity,THI_adj,HW,cum_HW,Temp15Threshold,Age
0,a624fb9a,SE-064c0cec-1189,5189,2022-01-01,06:25:00,7,191,13.90,2022-01-01 06:25:00,2022-1,...,NaN,NaN,2022-12-20,-3.025000,0.930917,28.012944,0,0,0,3095
1,a624fb9a,SE-064c0cec-1189,5189,2022-01-01,16:41:00,7,191,16.87,2022-01-01 16:41:00,2022-1,...,NaN,NaN,2022-12-20,-3.025000,0.930917,28.012944,0,0,0,3095
2,a624fb9a,SE-064c0cec-1189,5189,2022-01-02,15:29:00,7,192,20.41,2022-01-02 15:29:00,2022-1,...,NaN,NaN,2022-12-20,-0.279167,0.990542,32.898193,0,0,0,3096
3,a624fb9a,SE-064c0cec-1189,5189,2022-01-02,03:31:00,7,192,16.28,2022-01-02 03:31:00,2022-1,...,NaN,NaN,2022-12-20,-0.279167,0.990542,32.898193,0,0,0,3096
4,a624fb9a,SE-064c0cec-1189,5189,2022-01-02,22:44:00,7,192,11.53,2022-01-02 22:44:00,2022-1,...,NaN,NaN,2022-12-20,-0.279167,0.990542,32.898193,0,0,0,3096


In [31]:
# Define the THI threshold
THI_THRESHOLD = 61

# Calculate the daily heat load based on the THI threshold
data['HeatLoad'] = data['THI_adj'].apply(lambda x: x - THI_THRESHOLD if x > THI_THRESHOLD else -(THI_THRESHOLD - x))

# Initialize the cumulative heat load column with float type
data['CumulativeHeatLoad'] = 0.0  # Explicitly set as float

# Iterate through the data to calculate cumulative heat load correctly
for i in range(1, len(data)):
    previous_cumulative = data.at[i-1, 'CumulativeHeatLoad']
    current_heat_load = data.at[i, 'HeatLoad']
    if previous_cumulative + current_heat_load > 0:
        data.at[i, 'CumulativeHeatLoad'] = previous_cumulative + current_heat_load
    else:
        data.at[i, 'CumulativeHeatLoad'] = 0.0  # Ensure float is maintained

data.head(-5)

,FarmName_Pseudo,SE_Number,AnimalNumber,StartDate,StartTime,LactationNumber,DaysInMilk,TotalYield,DateTime,YearSeason,...,CullDecisionDate,Temperature,RelativeHumidity,THI_adj,HW,cum_HW,Temp15Threshold,Age,HeatLoad,CumulativeHeatLoad
0,a624fb9a,SE-064c0cec-1189,5189,2022-01-01,06:25:00,7,191,13.90,2022-01-01 06:25:00,2022-1,...,2022-12-20,-3.025000,0.930917,28.012944,0,0,0,3095,-32.987056,0.000000
1,a624fb9a,SE-064c0cec-1189,5189,2022-01-01,16:41:00,7,191,16.87,2022-01-01 16:41:00,2022-1,...,2022-12-20,-3.025000,0.930917,28.012944,0,0,0,3095,-32.987056,0.000000
2,a624fb9a,SE-064c0cec-1189,5189,2022-01-02,15:29:00,7,192,20.41,2022-01-02 15:29:00,2022-1,...,2022-12-20,-0.279167,0.990542,32.898193,0,0,0,3096,-28.101807,0.000000
3,a624fb9a,SE-064c0cec-1189,5189,2022-01-02,03:31:00,7,192,16.28,2022-01-02 03:31:00,2022-1,...,2022-12-20,-0.279167,0.990542,32.898193,0,0,0,3096,-28.101807,0.000000
4,a624fb9a,SE-064c0cec-1189,5189,2022-01-02,22:44:00,7,192,11.53,2022-01-02 22:44:00,2022-1,...,2022-12-20,-0.279167,0.990542,32.898193,0,0,0,3096,-28.101807,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1364930,f454e660,SE-fcdf259d-0044-0,1044,2023-06-07,00:51:00,10,351,5.63,2023-06-07 00:51:00,2023-3,...,NaT,15.645833,0.731917,61.559237,0,0,1,4154,0.559237,0.559237
1364931,f454e660,SE-fcdf259d-0044-0,1044,2023-06-07,11:17:00,10,351,3.34,2023-06-07 11:17:00,2023-3,...,NaT,15.645833,0.731917,61.559237,0,0,1,4154,0.559237,1.118475
1364932,f454e660,SE-fcdf259d-0044-0,1044,2023-06-08,17:01:00,10,352,6.96,2023-06-08 17:01:00,2023-3,...,NaT,15.570833,0.601708,59.383267,0,0,1,4155,-1.616733,0.000000
1364933,f454e660,SE-fcdf259d-0044-0,1044,2023-06-08,02:23:00,10,352,8.18,2023-06-08 02:23:00,2023-3,...,NaT,15.570833,0.601708,59.383267,0,0,1,4155,-1.616733,0.000000


In [32]:
# When CumulativeHeatLoad is greater than 5, it indicates that the cow is under heat stress
data['HeatStress'] = (data['CumulativeHeatLoad'] > 5).astype(int)
data.head(-5)

,FarmName_Pseudo,SE_Number,AnimalNumber,StartDate,StartTime,LactationNumber,DaysInMilk,TotalYield,DateTime,YearSeason,...,Temperature,RelativeHumidity,THI_adj,HW,cum_HW,Temp15Threshold,Age,HeatLoad,CumulativeHeatLoad,HeatStress
0,a624fb9a,SE-064c0cec-1189,5189,2022-01-01,06:25:00,7,191,13.90,2022-01-01 06:25:00,2022-1,...,-3.025000,0.930917,28.012944,0,0,0,3095,-32.987056,0.000000,0
1,a624fb9a,SE-064c0cec-1189,5189,2022-01-01,16:41:00,7,191,16.87,2022-01-01 16:41:00,2022-1,...,-3.025000,0.930917,28.012944,0,0,0,3095,-32.987056,0.000000,0
2,a624fb9a,SE-064c0cec-1189,5189,2022-01-02,15:29:00,7,192,20.41,2022-01-02 15:29:00,2022-1,...,-0.279167,0.990542,32.898193,0,0,0,3096,-28.101807,0.000000,0
3,a624fb9a,SE-064c0cec-1189,5189,2022-01-02,03:31:00,7,192,16.28,2022-01-02 03:31:00,2022-1,...,-0.279167,0.990542,32.898193,0,0,0,3096,-28.101807,0.000000,0
4,a624fb9a,SE-064c0cec-1189,5189,2022-01-02,22:44:00,7,192,11.53,2022-01-02 22:44:00,2022-1,...,-0.279167,0.990542,32.898193,0,0,0,3096,-28.101807,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1364930,f454e660,SE-fcdf259d-0044-0,1044,2023-06-07,00:51:00,10,351,5.63,2023-06-07 00:51:00,2023-3,...,15.645833,0.731917,61.559237,0,0,1,4154,0.559237,0.559237,0
1364931,f454e660,SE-fcdf259d-0044-0,1044,2023-06-07,11:17:00,10,351,3.34,2023-06-07 11:17:00,2023-3,...,15.645833,0.731917,61.559237,0,0,1,4154,0.559237,1.118475,0
1364932,f454e660,SE-fcdf259d-0044-0,1044,2023-06-08,17:01:00,10,352,6.96,2023-06-08 17:01:00,2023-3,...,15.570833,0.601708,59.383267,0,0,1,4155,-1.616733,0.000000,0
1364933,f454e660,SE-fcdf259d-0044-0,1044,2023-06-08,02:23:00,10,352,8.18,2023-06-08 02:23:00,2023-3,...,15.570833,0.601708,59.383267,0,0,1,4155,-1.616733,0.000000,0


In [33]:
# Calculate the DailyYield for each cow each day
data['DailyYield'] = data.groupby(['SE_Number', 'StartDate'])['TotalYield'].transform('sum')

# Sort the data by AnimalNumber and StartDate
data.sort_values(['AnimalNumber', 'StartDate'], inplace=True)

# Calculate the previous day's total yield for each cow
data['PreviousDailyYield'] = data.groupby('AnimalNumber')['DailyYield'].shift(1)

# Calculate the daily yield change for each cow
data['DailyYieldChange'] = data['DailyYield'] - data['PreviousDailyYield']

# Group and aggregate data
data = data.groupby(['SE_Number', 'FarmName_Pseudo', 'StartDate']).agg({
    'DailyYield': 'first',
    'PreviousDailyYield': 'first',
    'DailyYieldChange': 'first',
    'HW': 'max',
    'Temperature': 'mean',
    'THI_adj': 'mean',
    'DaysInMilk': 'first',
    'YearSeason': 'first',
    'cum_HW': 'max',
    'Temp15Threshold': 'max',
    'Age': 'first',
    'BreedName': 'first',
    'LactationNumber': 'first',
    'HeatLoad': 'mean',
    'CumulativeHeatLoad': 'mean',
    'HeatStress': 'max'
}).reset_index()

# Renaming and formatting
data.rename(columns={
    'Temperature': 'MeanTemperature',
    'THI_adj': 'MeanTHI_adj',
    'StartDate': 'Date'
}, inplace=True)
data['Date'] = pd.to_datetime(data['Date'])

# Display the first few rows of the transformed data
data.head()

,SE_Number,FarmName_Pseudo,Date,DailyYield,PreviousDailyYield,DailyYieldChange,HW,MeanTemperature,MeanTHI_adj,DaysInMilk,YearSeason,cum_HW,Temp15Threshold,Age,BreedName,LactationNumber,HeatLoad,CumulativeHeatLoad,HeatStress
0,SE-064c0cec-1189,a624fb9a,2022-01-01,30.77,30.77,0.00,0,-3.025000,28.012944,191,2022-1,0,0,3095,02 SLB,7,-32.987056,0.0,0
1,SE-064c0cec-1189,a624fb9a,2022-01-02,48.22,30.77,17.45,0,-0.279167,32.898193,192,2022-1,0,0,3096,02 SLB,7,-28.101807,0.0,0
2,SE-064c0cec-1189,a624fb9a,2022-01-03,30.53,48.22,-17.69,0,2.033333,36.760487,193,2022-1,0,0,3097,02 SLB,7,-24.239513,0.0,0
3,SE-064c0cec-1189,a624fb9a,2022-01-04,42.26,30.53,11.73,0,0.066667,31.939524,194,2022-1,0,0,3098,02 SLB,7,-29.060476,0.0,0
4,SE-064c0cec-1189,a624fb9a,2022-01-05,38.49,42.26,-3.77,0,-3.700000,26.498206,195,2022-1,0,0,3099,02 SLB,7,-34.501794,0.0,0


In [34]:
# Check if DailyYield is centered around approx the same for each farm
print("Mean of DailyYield:", data.groupby('FarmName_Pseudo')['DailyYield'].mean())
print("Standard Deviation of DailyYield:", data.groupby('FarmName_Pseudo')['DailyYield'].std())

Mean of DailyYield: FarmName_Pseudo
5c06d92d    37.389675
752efd72    31.151716
a624fb9a    33.413694
f454e660    30.485127
Name: DailyYield, dtype: float64
Standard Deviation of DailyYield: FarmName_Pseudo
5c06d92d     9.960240
752efd72     7.799288
a624fb9a    11.050811
f454e660    11.833056
Name: DailyYield, dtype: float64


In [35]:
# Define the Wilmink Lactation Curve function
def wilmink_lactation_curve(dim, a, b, c, d):
    dim = np.array(dim, dtype=float)
    return a + b * dim + c * np.exp(-d * dim)

# Function to remove outliers
def remove_outliers(group, threshold=3.5):
    mean = np.mean(group['DailyYield'])
    std_dev = np.std(group['DailyYield'])
    return group[(group['DailyYield'] > mean - threshold * std_dev) & (group['DailyYield'] < mean + threshold * std_dev)]

# Function to smooth the data
def smooth_data(group, window=5):
    group = group.copy()
    group['DailyYield'] = group['DailyYield'].rolling(window, min_periods=1).mean()
    return group

# Quantile loss function
def quantile_loss(params, dim, yield_data_quantile):
    a, b, c, d = params
    predictions = wilmink_lactation_curve(dim, a, b, c, d)
    residuals = yield_data_quantile - predictions
    loss = np.sum(np.abs(residuals))
    return loss

# Function to fit the Wilmink curve using VQR for quantile estimation
def fit_wilmink_lactation_curve_with_vqr(dataset, quantile=0.7):
    dataset['ExpectedYield'] = np.nan
    params_dict = {}

    vqr_solver = RegularizedDualVQRSolver(verbose=False, epsilon=1e-2, num_epochs=1000, lr=0.9)
    vqr = VectorQuantileRegressor(solver=vqr_solver)

    for (animal_number, lactation_number), group in tqdm(dataset.groupby(['SE_Number', 'LactationNumber']), unit=" Segments"):
        group = remove_outliers(group)
        group = smooth_data(group)
        x_data = group['DaysInMilk'].values.reshape(-1, 1).astype(float)
        y_data = group['DailyYield'].values.reshape(-1, 1).astype(float)

        if len(x_data) < 10 or len(y_data) < 10:
            continue

        try:
            vqr.fit(x_data, y_data)
            all_quantiles = vqr.predict(x_data)
            print(f"all_quantiles shape: {all_quantiles.shape}")  # Debugging: check the shape of the output
            print(f"all_quantiles content: {all_quantiles}")  # Debugging: print the content of the output

            # Squeeze out the unnecessary dimension
            all_quantiles = np.squeeze(all_quantiles, axis=1)
            print(f"all_quantiles after squeezing shape: {all_quantiles.shape}")  # Check shape after squeezing

            # Expecting two dimensions: (n_samples, n_quantiles)
            quantile_index = int(quantile * (all_quantiles.shape[1] - 1))
            y_data_quantile = all_quantiles[:, quantile_index]

            initial_guesses = [np.mean(y_data), 0, np.mean(y_data) / 2, 0.1]
            bounds = ([-np.inf, -np.inf, -np.inf, 0], [np.inf, np.inf, np.inf, np.inf])

            result = minimize(
                quantile_loss, initial_guesses, args=(x_data.flatten(), y_data_quantile),
                method='L-BFGS-B', bounds=bounds
            )

            if result.success:
                popt = result.x
                params_dict[(animal_number, lactation_number)] = {'a': popt[0], 'b': popt[1], 'c': popt[2], 'd': popt[3]}
                dataset.loc[group.index, 'ExpectedYield'] = wilmink_lactation_curve(group['DaysInMilk'], *popt)
            else:
                print(f"Optimization failed for cow {animal_number}, lactation {lactation_number}.")
        except Exception as e:
            print(f"Error with cow {animal_number}, lactation {lactation_number}: {e}")

    dataset['ExpectedYield'] = dataset['ExpectedYield'].fillna(0)
    return dataset, params_dict

# Apply the quantile-based curve fitting function
data, params_dict = fit_wilmink_lactation_curve_with_vqr(data, quantile=0.7)

  0%|          | 0/2315 [00:00<?, ? Segments/s]/Users/user/anaconda3/envs/GIGACOW/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
  0%|          | 1/2315 [00:00<15:44,  2.45 Segments/s]

all_quantiles shape: (88, 1, 50)
all_quantiles content: [[[31.325445 31.325445 32.783497 ... 39.182472 39.362144 39.488407]]

 [[31.214529 31.214529 32.6622   ... 39.056587 39.236065 39.36138 ]]

 [[31.10361  31.10361  32.540894 ... 38.93051  39.109802 39.234543]]

 ...

 [[21.897303 21.897303 22.473217 ... 28.48501  28.630445 28.701971]]

 [[21.675467 21.675467 22.230625 ... 28.233337 28.377913 28.4482  ]]

 [[21.453619 21.453619 21.988035 ... 27.981565 28.125477 28.19433 ]]]
all_quantiles after squeezing shape: (88, 50)
Error with cow SE-064c0cec-1189, lactation 7: too many values to unpack (expected 2)


/Users/user/anaconda3/envs/GIGACOW/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
  0%|          | 2/2315 [00:00<14:51,  2.59 Segments/s]

all_quantiles shape: (207, 1, 50)
all_quantiles content: [[[18.076801 18.076801 22.384094 ... 43.85242  44.145966 44.439507]]

 [[18.096209 18.096209 22.386694 ... 43.769264 44.061466 44.353867]]

 [[18.115604 18.115604 22.389294 ... 43.686485 43.976974 44.268036]]

 ...

 [[22.035818 22.035818 22.913551 ... 26.916216 26.921558 26.942825]]

 [[22.055225 22.055225 22.91615  ... 26.833248 26.837156 26.856995]]

 [[22.074627 22.074627 22.918747 ... 26.750183 26.752758 26.77126 ]]]
all_quantiles after squeezing shape: (207, 50)
Error with cow SE-064c0cec-1189, lactation 8: too many values to unpack (expected 2)


/Users/user/anaconda3/envs/GIGACOW/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
  0%|          | 3/2315 [00:01<12:43,  3.03 Segments/s]

...

/Users/user/anaconda3/envs/GIGACOW/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
 22%|██▏       | 509/2315 [03:10<10:53,  2.76 Segments/s]

all_quantiles shape: (118, 1, 50)
all_quantiles content: [[[29.168783 29.168783 28.868372 ... 33.824825 33.600044 33.081818]]

 [[29.148922 29.148922 28.86347  ... 33.828068 33.61206  33.110046]]

 [[29.129076 29.129076 28.858578 ... 33.83131  33.624363 33.138084]]

 ...

 [[26.8854   26.8854   28.305674 ... 34.198284 34.994793 36.32927 ]]

 [[26.865541 26.865541 28.300774 ... 34.20143  35.007    36.357403]]

 [[26.865541 26.865541 28.300774 ... 34.20143  35.007    36.357403]]]
all_quantiles after squeezing shape: (118, 50)
Error with cow SE-5c06d92d-3080, lactation 1: too many values to unpack (expected 2)


/Users/user/anaconda3/envs/GIGACOW/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
 22%|██▏       | 510/2315 [03:11<11:34,  2.60 Segments/s]/Users/user/anaconda3/envs/GIGACOW/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
 22%|██▏       | 510/2315 [03:11<11:17,  2.66 Segments/s]

all_quantiles shape: (499, 1, 50)
all_quantiles content: [[[21.605646 21.605646 30.408085 ... 51.257893 51.63288  51.955605]]

 [[21.607256 21.607256 30.452633 ... 51.37272  51.748844 52.07214 ]]

 [[21.605646 21.605646 30.408085 ... 51.257893 51.63288  51.955605]]

 ...

 [[21.316242 21.316242 22.368393 ... 30.51691  30.703926 30.929089]]

 [[21.316242 21.316242 22.368393 ... 30.51691  30.703926 30.929089]]

 [[21.316242 21.316242 22.368393 ... 30.51691  30.703926 30.929089]]]
all_quantiles after squeezing shape: (499, 50)
Error with cow SE-5c06d92d-3080, lactation 2: too many values to unpack (expected 2)


KeyboardInterrupt: 